In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import torch
import torch.nn as nn
import torchbnn as bnn

In [ ]:
# Load Data
colectomy_cci_var = pd.read_csv('./colectomy_cci_var.csv', dtype = 'str')

# Select Relevant Features
features = ['year', 'ptsex', 'race', 'age', 'adtype', 'adsource', 'admdx', 'sdx1', 'sdx2']

#colectomy_cci_var['ptsex'] = colectomy_cci_var['ptsex'].replace({'F': 'female', 'M': 'male'})
#colectomy_cci_var['adtype'] = colectomy_cci_var['adtype'].replace({1: 'emergency', 2: 'urgent', 3: 'elective'})
#colectomy_cci_var['adsource'] = colectomy_cci_var['adsource'].replace({'F': 'female', 'M': 'male'})

colectomy_cci_var_X = colectomy_cci_var[features]
colectomy_cci_var_y = colectomy_cci_var['pdx']

for feature in features:
    print(colectomy_cci_var_X[feature].value_counts())

colectomy_cci_var_X_encoded = pd.get_dummies(colectomy_cci_var_X, columns = ['ptsex', 'race', 'adtype', 'adsource', 'admdx', 'sdx1', 'sdx2'])

# Split Data
train_X, test_X, train_y, test_y = train_test_split(colectomy_cci_var_X_encoded, colectomy_cci_var_y, test_size = 0.2, random_state = 0)

# Normalize Data
train_X

In [ ]:
k = 1
print(len(colectomy_cci_var))
colectomy_cci_var_filtered_1 = colectomy_cci_var.groupby('admdx').filter(lambda x: len(x) >= k)
print(len(colectomy_cci_var_filtered_1))
colectomy_cci_var_filtered_2 = colectomy_cci_var_filtered_1.groupby('sdx1').filter(lambda x: len(x) >= k)
print(len(colectomy_cci_var_filtered_2))
colectomy_cci_var_filtered_3 = colectomy_cci_var_filtered_2.groupby('sdx2').filter(lambda x: len(x) >= k)
print(len(colectomy_cci_var_filtered_3))

In [ ]:
# Model Architecture
class BayesianNeuralNetwork(nn.Module):
    def __init__(self, feature_size, num_classes):
        super().__init__()

        layer_1_size = 100
        layer_2_size = 50

        # Layers
        self.bayes_linear_1 = bnn.BayesLinear(prior_mu = 0, prior_sigma = 1, in_features = feature_size, out_features = layer_1_size)
        self.bayes_linear_2 = bnn.BayesLinear(prior_mu = 0, prior_sigma = 1, in_features = layer_1_size, out_features = layer_2_size)
        self.output_layer = nn.Linear(in_features = layer_2_size, out_features = num_classes)
    
    def forward(self, x):
        x = torch.relu(self.bayes_linear_1(x))
        x = torch.relu(self.bayes_linear_2(x))
        return self.output_layer(x)

In [ ]:
# Loss Function
def elbo_loss(output, target, model):
    standard_loss = nn.CrossEntropyLoss(output, target)
    kl_loss = bnn.BKLLoss(reduction = 'mean', last_layer_only = False)

    return standard_loss + kl_loss(model)